In [ ]:
from google.colab import drive
drive.mount('/content/drive')

ValueError: mount failed

In [ ]:
import zipfile
import os

base_path = '/content/drive/MyDrive/Banco com zip/'

for year in range(2000, 2025):
    zip_file_path = os.path.join(base_path, str(year) + '.zip')
    if os.path.exists(zip_file_path):
        with zipfile.ZipFile(zip_file_path, 'r') as zip_ref:
            extraction_path = os.path.join('diretorio_destino', str(year))
            zip_ref.extractall(extraction_path)
            print(f"Files from {zip_file_path} extracted to {extraction_path}")
    else:
        print(f"Zip file not found for year {year}: {zip_file_path}")

Files from /content/drive/MyDrive/Banco com zip/2000.zip extracted to diretorio_destino/2000
Files from /content/drive/MyDrive/Banco com zip/2001.zip extracted to diretorio_destino/2001
Files from /content/drive/MyDrive/Banco com zip/2002.zip extracted to diretorio_destino/2002
Files from /content/drive/MyDrive/Banco com zip/2003.zip extracted to diretorio_destino/2003
Files from /content/drive/MyDrive/Banco com zip/2004.zip extracted to diretorio_destino/2004
Files from /content/drive/MyDrive/Banco com zip/2005.zip extracted to diretorio_destino/2005
Files from /content/drive/MyDrive/Banco com zip/2006.zip extracted to diretorio_destino/2006
Files from /content/drive/MyDrive/Banco com zip/2007.zip extracted to diretorio_destino/2007
Files from /content/drive/MyDrive/Banco com zip/2008.zip extracted to diretorio_destino/2008
Files from /content/drive/MyDrive/Banco com zip/2009.zip extracted to diretorio_destino/2009
Files from /content/drive/MyDrive/Banco com zip/2010.zip extracted to 

In [ ]:
import os
import polars as pl
import sqlite3

def processar_e_salvar_no_sql(ano, chunk_size=1000):
    db_path = '/content/dados_anos.db'
    conn = sqlite3.connect(db_path)
    cursor = conn.cursor()

    diretorio_destino = f'/content/diretorio_destino/{ano}/{ano}' if ano < 2020 else f'/content/diretorio_destino/{ano}'
    arquivos_csv = [f for f in os.listdir(diretorio_destino) if f.lower().endswith(".csv")]

    tabela_criada = False  # flag pra ter ctz que a tabela seja criada só uma vez

    for arquivo in arquivos_csv:
        caminho_completo = os.path.join(diretorio_destino, arquivo)
        estacao = arquivo.split('_')[3]

        # capturar as colunas
        colunas = pl.read_csv(
            caminho_completo,
            skip_rows=8,
            n_rows=1,
            separator=";",
            decimal_comma=True,
            encoding="ISO-8859-1",
            infer_schema_length=0
        ).columns

        # leitura e inserção em chunks para econnmizar ram
        for chunk_df in pl.read_csv(
                caminho_completo,
                skip_rows=8,
                separator=";",
                decimal_comma=True,
                encoding="ISO-8859-1",
                infer_schema_length=10000,
                ignore_errors=True,
                null_values="-9999",
        ).iter_slices(chunk_size):
            chunk_df = chunk_df.with_columns(pl.lit(estacao).alias("estacao"))

            if not tabela_criada:
                cursor.execute(f"DROP TABLE IF EXISTS dados_{ano}")
                colunas_sql = ", ".join([f'"{col}" TEXT' for col in chunk_df.columns])
                cursor.execute(f"CREATE TABLE dados_{ano} ({colunas_sql})")
                conn.commit()
                tabela_criada = True

            registros = chunk_df.to_numpy().tolist()
            placeholders = ', '.join(['?' for _ in chunk_df.columns])

            cursor.executemany(
                f'INSERT INTO dados_{ano} VALUES ({placeholders})',
                registros
            )

    conn.commit()
    conn.close()
    print(f"Dados do ano {ano} processados e salvos no banco de dados com sucesso!")

for ano in range(2000, 2025):
    processar_e_salvar_no_sql(ano, chunk_size=1000)
    print(f"Data para o ano {ano} processada e armazenada com sucesso!")



Dados do ano 2000 processados e salvos no banco de dados com sucesso!
Data para o ano 2000 processada e armazenada com sucesso!
Dados do ano 2001 processados e salvos no banco de dados com sucesso!
Data para o ano 2001 processada e armazenada com sucesso!
Dados do ano 2002 processados e salvos no banco de dados com sucesso!
Data para o ano 2002 processada e armazenada com sucesso!
Dados do ano 2003 processados e salvos no banco de dados com sucesso!
Data para o ano 2003 processada e armazenada com sucesso!
Dados do ano 2004 processados e salvos no banco de dados com sucesso!
Data para o ano 2004 processada e armazenada com sucesso!
Dados do ano 2005 processados e salvos no banco de dados com sucesso!
Data para o ano 2005 processada e armazenada com sucesso!
Dados do ano 2006 processados e salvos no banco de dados com sucesso!
Data para o ano 2006 processada e armazenada com sucesso!
Dados do ano 2007 processados e salvos no banco de dados com sucesso!
Data para o ano 2007 processada e 

In [ ]:
db_path = '/content/dados_anos.db'
conn = sqlite3.connect(db_path)

In [ ]:
import pandas as pd

In [ ]:
pd.read_sql_query("""SELECT * FROM dados_2024
WHERE estacao == 'A717'""", conn)

,Data,Hora UTC,"PRECIPITAÇÃO TOTAL, HORÁRIO (mm)","PRESSAO ATMOSFERICA AO NIVEL DA ESTACAO, HORARIA (mB)",PRESSÃO ATMOSFERICA MAX.NA HORA ANT. (AUT) (mB),PRESSÃO ATMOSFERICA MIN. NA HORA ANT. (AUT) (mB),RADIACAO GLOBAL (Kj/m²),"TEMPERATURA DO AR - BULBO SECO, HORARIA (°C)",TEMPERATURA DO PONTO DE ORVALHO (°C),TEMPERATURA MÁXIMA NA HORA ANT. (AUT) (°C),...,TEMPERATURA ORVALHO MAX. NA HORA ANT. (AUT) (°C),TEMPERATURA ORVALHO MIN. NA HORA ANT. (AUT) (°C),UMIDADE REL. MAX. NA HORA ANT. (AUT) (%),UMIDADE REL. MIN. NA HORA ANT. (AUT) (%),"UMIDADE RELATIVA DO AR, HORARIA (%)","VENTO, DIREÇÃO HORARIA (gr) (° (gr))","VENTO, RAJADA MAXIMA (m/s)","VENTO, VELOCIDADE HORARIA (m/s)",,estacao
0,2024/01/01,0000 UTC,0.0,997.5,997.5,996.9,None,28.0,23.9,29.6,...,25.3,23.9,80.0,75.0,79.0,73.0,6.0,2.8,None,A717
1,2024/01/01,0100 UTC,0.0,998.3,998.3,997.5,None,26.9,23.8,28.0,...,24.0,23.6,83.0,79.0,83.0,77.0,4.7,2.3,None,A717
2,2024/01/01,0200 UTC,0.0,999.4,999.4,998.3,None,26.1,23.3,26.9,...,23.7,23.2,85.0,82.0,85.0,70.0,3.9,2.5,None,A717
3,2024/01/01,0300 UTC,0.0,999.2,999.5,999.2,None,25.6,23.1,26.1,...,23.3,23.0,86.0,85.0,86.0,70.0,5.2,2.7,None,A717
4,2024/01/01,0400 UTC,0.0,998.8,999.3,998.7,None,25.3,23.2,25.7,...,23.2,23.0,88.0,85.0,88.0,79.0,4.5,1.7,None,A717
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7315,2024/10/31,1900 UTC,0.0,991.9,993.1,991.9,1389.2,35.3,16.2,37.2,...,18.3,16.0,35.0,32.0,32,2,10.5,4.3,None,A717
7316,2024/10/31,2000 UTC,0.0,990.8,991.9,990.8,889.7,35.6,17.6,35.7,...,17.9,16.1,36.0,32.0,34,13,6.4,1.9,None,A717
7317,2024/10/31,2100 UTC,0.0,991.1,991.1,990.8,594.9,35.2,18.3,36.0,...,19.4,17.0,40.0,33.0,37,58,3.2,1.2,None,A717
7318,2024/10/31,2200 UTC,0.0,991.7,991.7,991.1,122.5,32.1,21.0,35.3,...,22.2,18.0,55.0,36.0,52,117,1.6,0.3,None,A717


In [ ]:
pd.read_sql_query("""SELECT Data, "Hora UTC",`TEMPERATURA DO AR - BULBO SECO, HORARIA (°C)`, estacao FROM dados_2024
ORDER BY `TEMPERATURA DO AR - BULBO SECO, HORARIA (°C)` DESC LIMIT 1""", conn)

,Data,Hora UTC,"TEMPERATURA DO AR - BULBO SECO, HORARIA (°C)",estacao
0,2024/05/18,1100 UTC,9.9,B807


In [ ]:
import os
import polars as pl
import sqlite3

def map_polars_to_sql_type(polars_type):
    if polars_type == pl.Int32 or polars_type == pl.Int64:
        return "INTEGER"
    elif polars_type == pl.Float32 or polars_type == pl.Float64:
        return "REAL"
    else:
        return "TEXT"

def processar_e_salvar_no_sql(ano, chunk_size=1000):
    db_path = '/content/dados_anos.db'
    conn = sqlite3.connect(db_path)
    cursor = conn.cursor()

    diretorio_destino = f'/content/diretorio_destino/{ano}/{ano}' if ano < 2020 else f'/content/diretorio_destino/{ano}'

    arquivos_csv = [f for f in os.listdir(diretorio_destino) if f.lower().endswith(".csv")]

    tabela_criada = False 

    for arquivo in arquivos_csv:
        caminho_completo = os.path.join(diretorio_destino, arquivo)
        estacao = arquivo.split('_')[3]

        first_chunk = pl.read_csv(
            caminho_completo,
            skip_rows=8,
            separator=";",
            decimal_comma=True,
            encoding="ISO-8859-1",
            infer_schema_length=10000
        ).with_columns(pl.lit(estacao).alias("estacao"))

        colunas = first_chunk.columns
        tipos = [map_polars_to_sql_type(dtype) for dtype in first_chunk.dtypes]

        if not tabela_criada:
            cursor.execute(f"DROP TABLE IF EXISTS dados_{ano}")

            colunas_sql = ", ".join([f'"{col}" {tipo}' for col, tipo in zip(colunas, tipos)])
            cursor.execute(f"CREATE TABLE dados_{ano} ({colunas_sql})")
            conn.commit()
            tabela_criada = True

        for chunk_df in pl.read_csv(
                caminho_completo,
                skip_rows=8,
                separator=";",
                decimal_comma=True,
                encoding="ISO-8859-1",
                infer_schema_length=10000,
                ignore_errors=True,
                null_values="-9999",
        ).iter_slices(chunk_size):
            chunk_df = chunk_df.with_columns(pl.lit(estacao).alias("estacao"))
            registros = chunk_df.to_numpy().tolist()
            placeholders = ', '.join(['?' for _ in colunas])

            cursor.executemany(
                f'INSERT INTO dados_{ano} VALUES ({placeholders})',
                registros
            )

    conn.commit()
    conn.close()
    print(f"Dados do ano {ano} processados e salvos no banco de dados com sucesso!")

for ano in range(2000, 2025):
    processar_e_salvar_no_sql(ano, chunk_size=1000)
    print(f"Data para o ano {ano} processada e armazenada com sucesso!")


Dados do ano 2000 processados e salvos no banco de dados com sucesso!
Data para o ano 2000 processada e armazenada com sucesso!
Dados do ano 2001 processados e salvos no banco de dados com sucesso!
Data para o ano 2001 processada e armazenada com sucesso!
Dados do ano 2002 processados e salvos no banco de dados com sucesso!
Data para o ano 2002 processada e armazenada com sucesso!
Dados do ano 2003 processados e salvos no banco de dados com sucesso!
Data para o ano 2003 processada e armazenada com sucesso!
Dados do ano 2004 processados e salvos no banco de dados com sucesso!
Data para o ano 2004 processada e armazenada com sucesso!
Dados do ano 2005 processados e salvos no banco de dados com sucesso!
Data para o ano 2005 processada e armazenada com sucesso!
Dados do ano 2006 processados e salvos no banco de dados com sucesso!
Data para o ano 2006 processada e armazenada com sucesso!
Dados do ano 2007 processados e salvos no banco de dados com sucesso!
Data para o ano 2007 processada e 

In [ ]:
import pandas as pd

In [ ]:
db_path = '/content/dados_anos.db'
conn = sqlite3.connect(db_path)

In [ ]:
pd.read_sql_query("""SELECT * FROM dados_2000""", conn)

,DATA (YYYY-MM-DD),HORA (UTC),"PRECIPITAÇÃO TOTAL, HORÁRIO (mm)","PRESSAO ATMOSFERICA AO NIVEL DA ESTACAO, HORARIA (mB)",PRESSÃO ATMOSFERICA MAX.NA HORA ANT. (AUT) (mB),PRESSÃO ATMOSFERICA MIN. NA HORA ANT. (AUT) (mB),RADIACAO GLOBAL (KJ/m²),"TEMPERATURA DO AR - BULBO SECO, HORARIA (°C)",TEMPERATURA DO PONTO DE ORVALHO (°C),TEMPERATURA MÁXIMA NA HORA ANT. (AUT) (°C),...,TEMPERATURA ORVALHO MAX. NA HORA ANT. (AUT) (°C),TEMPERATURA ORVALHO MIN. NA HORA ANT. (AUT) (°C),UMIDADE REL. MAX. NA HORA ANT. (AUT) (%),UMIDADE REL. MIN. NA HORA ANT. (AUT) (%),"UMIDADE RELATIVA DO AR, HORARIA (%)","VENTO, DIREÇÃO HORARIA (gr) (° (gr))","VENTO, RAJADA MAXIMA (m/s)","VENTO, VELOCIDADE HORARIA (m/s)",,estacao
0,2000-09-22,00:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,None,A801
1,2000-09-22,01:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,None,A801
2,2000-09-22,02:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,None,A801
3,2000-09-22,03:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,None,A801
4,2000-09-22,04:00,0.0,1013.9,1014.2,1013.2,NaN,15.5,14.6,15.6,...,14.8,14.6,95.0,94.0,94.0,231.0,4.4,1.2,None,A801
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
25171,2000-12-31,19:00,0.0,1003.1,1004.5,1003.1,1154.0,27.4,23.3,28.1,...,24.1,23.0,82.0,75.0,78.0,209.0,2.7,0.7,None,A101
25172,2000-12-31,20:00,0.0,1002.3,1003.1,1002.3,589.0,26.8,23.2,27.4,...,23.6,23.0,82.0,78.0,81.0,251.0,2.9,1.7,None,A101
25173,2000-12-31,21:00,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,None,A101
25174,2000-12-31,22:00,0.0,1002.9,1002.9,1002.4,210.0,26.0,23.3,26.7,...,23.5,23.1,85.0,81.0,85.0,247.0,3.6,1.4,None,A101


In [ ]:
pd.read_sql_query("""SELECT "DATA (YYYY-MM-DD)", `HORA (UTC)`,`TEMPERATURA DO AR - BULBO SECO, HORARIA (°C)`, estacao FROM dados_2000
ORDER BY `TEMPERATURA DO AR - BULBO SECO, HORARIA (°C)` DESC LIMIT 1""", conn)

,DATA (YYYY-MM-DD),HORA (UTC),"TEMPERATURA DO AR - BULBO SECO, HORARIA (°C)",estacao
0,2000-10-12,18:00,38.2,A601
